In [2]:
# 1. 파이썬 코드에서 Matplotlib 폰트 설정
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import timedelta
from itertools import combinations
from collections import Counter

# 폰트 설정
plt.rc('font', family='Malgun Gothic')
# 마이너스 부호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

print("한글 폰트 설정이 완료되었습니다.")


한글 폰트 설정이 완료되었습니다.


In [3]:
# 3. 데이터 로드 및 통합 (모든 문제 풀이의 시작점)
# dataframe 확인해라!
try:
    orders_df = pd.read_csv('../data/orders.csv')
    payments_df = pd.read_csv('../data/payments.csv')
    products_df = pd.read_csv('../data/products.csv')
    shipping_df = pd.read_csv('../data/shipping.csv')
    customers_df = pd.read_csv('../data/customers.csv')

    # 모든 데이터프레임 병합
    df = pd.merge(orders_df, payments_df, on='order_id', how='left')
    df = pd.merge(df, products_df, on='product_id', how='left')
    df = pd.merge(df, customers_df, on='customer_id', how='left')
    df = pd.merge(df, shipping_df, on='order_id', how='left')

    # 데이터 전처리
    date_cols = ['order_date', 'payment_date', 'join_date', 'shipping_start_date', 'shipping_end_date']
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    df['total_sales'] = df['quantity'] * df['price']
    
    print("데이터 로드 및 통합이 완료되었습니다.")

except FileNotFoundError as e:
    print(f"파일을 찾을 수 없습니다: {e}")


데이터 로드 및 통합이 완료되었습니다.


In [ ]:
# 문제 4: 카테고리 내에서 각 상품의 매출액 순위를 계산하고, 카테고리별 상위 3개 상품을 확인하세요.
# 비즈니스 목적: 각 카테고리의 핵심 상품(Key Item)과 비주력 상품을 파악하여 상품 진열, 프로모션, 재고 관리에 활용합니다.

In [ ]:
# 출력 결과를 보고 코딩하세요

카테고리별 매출 상위 3개 상품:
    category  product_name  total_sales  rank
4         가구     가구 상품_382      6860700   1.0
44        가구   전자제품 상품_273      6623000   2.0
57        가구    화장품 상품_323      6123000   3.0
70        도서     도서 상품_360      7355400   1.0
74        도서      도서 상품_83      6828800   2.0
112       도서     화장품 상품_82      6683000   3.0
141    스포츠용품   스포츠용품 상품_61      8710000   1.0
146    스포츠용품     식품 상품_173      7522900   2.0
151    스포츠용품     식품 상품_473      7260100   3.0
198       식품      가구 상품_46      9682200   1.0
216       식품  스포츠용품 상품_482      6859800   2.0
238       식품   전자제품 상품_280      6697500   3.0
344       의류   전자제품 상품_463      8091900   1.0
315       의류     식품 상품_396      7265700   2.0
347       의류    화장품 상품_224      6907500   3.0
377     전자제품     도서 상품_391      9197700   1.0
399     전자제품     의류 상품_145      8109000   2.0
413     전자제품   전자제품 상품_218      7281600   3.0
458      화장품  스포츠용품 상품_376      8261600   1.0
469      화장품     식품 상품_329      7087600   2.0
434      화장품   

In [90]:
# 출력 결과를 보고 코딩하세요
# 문제 : 카테고리 내에서 각 상품의 매출액 순위를 계산하고, 카테고리별 상위 3개 상품을 확인
# 출력항목   category  product_name  total_sales  rank
# df.columns
''' 
Index(['order_id', 'customer_id', 'product_id', 'order_date', 'quantity',
       'payment_id', 'payment_method', 'payment_status', 'payment_date',
       'product_name', 'category', 'price', 'stock', 'name', 'gender', 'age',
       'join_date', 'city', 'shipping_id', 'shipping_company',
       'shipping_status', 'shipping_start_date', 'shipping_end_date',
       'total_sales', 'year_month'],
      dtype='object')
'''
# product_id 기준으로 카테고리별 상위 3개 상품 확인
# 1. 카테고리별, 상품ID별 총 매출액 집계해서 DataFrame 만들기
category_product_sales = df.groupby(['category', 'product_id'])['total_sales'].sum().reset_index()


# 2. 매출액 데이터에 상품명 조인 및 순위 계산
results = (
    category_product_sales
    # 상품명을 가져오기 위해 product_id와 join
    .merge(products_df[['product_id', 'product_name']], on='product_id', how='left') 
    # 카테고리별 총 매출액으로 그룹핑한 후 rank()함수를 이용해 ranking구하고
    .assign(rank=lambda x: x.groupby('category')['total_sales'].rank(method='dense', ascending=False))
    # 카테고리별 랭킹으로 sort하고
    .sort_values(['category', 'rank'])
    # 랭킹이 top3 인 것들만 가져와서 
    .query('rank <= 3')[['category', 'product_name', 'total_sales', 'rank']]
)

print("카테고리별 상위 3개 상품:")
print(results)

카테고리별 상위 3개 상품:
    category  product_name  total_sales  rank
49        가구     가구 상품_382      6860700   1.0
39        가구   전자제품 상품_273      6623000   2.0
46        가구    화장품 상품_323      6123000   3.0
97        도서     도서 상품_360      7355400   1.0
68        도서      도서 상품_83      6828800   2.0
67        도서     화장품 상품_82      6683000   3.0
118    스포츠용품   스포츠용품 상품_61      8710000   1.0
132    스포츠용품     식품 상품_173      7522900   2.0
187    스포츠용품     식품 상품_473      7260100   3.0
199       식품      가구 상품_46      9682200   1.0
260       식품  스포츠용품 상품_482      6859800   2.0
230       식품   전자제품 상품_280      6697500   3.0
352       의류   전자제품 상품_463      8091900   1.0
337       의류     식품 상품_396      7265700   2.0
305       의류    화장품 상품_224      6907500   3.0
409     전자제품     도서 상품_391      9197700   1.0
374     전자제품     의류 상품_145      8109000   2.0
387     전자제품   전자제품 상품_218      7281600   3.0
482      화장품  스포츠용품 상품_376      8261600   1.0
475      화장품     식품 상품_329      7087600   2.0
462      화장품     가